```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec


import os
import pyopencl
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import CTCS, CDKLM16, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *
from SWESimulators import DataAssimilationUtils as dautils

from SWESimulators import WindForcingEnsemble

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

# The Lorenz Attractor (Lorenz-63)

Rules of thumb:
- observations every 8th timestep: quasi-linear problem
- observations every 25th timestep: full non-linear problem

In [ ]:

class Lorenz63Ensemble:
        
    def __init__(self, cl_ctx, numParticles, \
                 observation_variance=np.sqrt(2), model_error=np.sqrt(2)*0.01, 
                 init_error=np.sqrt(2), obs_vars='111', \
                 T=10):

        self.T = T
        
        self.numParticles = numParticles
        self.particles = [None]*(self.numParticles + 1)
        
        self.obs_index = self.numParticles
                
        self.observation_variance = np.sqrt(observation_variance)
        self.model_error = np.sqrt(model_error)
        self.init_error = np.sqrt(init_error)
        
        # Use parameters from the PJvL-2010 paper
        self.dt = 0.01
        self.sigma = 10
        self.rho = 28
        self.beta = 8.0/3.0
        self.initValues = [1.508870, -1.531271, 25.46091]
            
        self.timestep = 0
        self.t = 0.0
        
        self.init()
        self.observations = []
        self.obs_vars = obs_vars
        self.obs_matrix = self._create_observation_matrix()
        self.observation_size = str.count(obs_vars, '1')

        
    def _create_observation_matrix(self):
        obs_matrix_array = []
        if self.obs_vars[0]=='1':
            obs_matrix_array.append([1, 0, 0])
        if self.obs_vars[1]=='1':
            obs_matrix_array.append([0,1,0])
        if self.obs_vars[2]=='1':
            obs_matrix_array.append([0,0,1])
        return  np.matrix(obs_matrix_array)
    
    def _observe(self, fullState):
        obs = []
        for i in range(len(fullState)):
            if self.obs_vars[i]=='1':
                obs.append(fullState[i])
        return np.array(obs)

    # ---------------------------------------
    # Override functions we don't need.
    # ---------------------------------------       
    def cleanUp(self):
        pass
        
    def setGridInfo(self, nx, ny, dx, dy, dt, boundaryConditions, eta=None, hu=None, hv=None, H=None):
        pass
    
    def setParameters(self, f=0, g=9.81, beta=0, r=0, wind=None):
        pass
    
    
    # ---------------------------------------
    # Needs to be abstract!
    # ---------------------------------------
    def init(self):
        self.max_timesteps = self.T+1
        for p in range(self.numParticles+1):
            self.particles[p] = np.zeros((3, self.max_timesteps))
            self.particles[p][:,0] = np.random.multivariate_normal(self.initValues, np.eye(3)*self.init_error )
    
    
    def observeParticles(self, all=False):
        positions=None
        if all:
            positions = np.zeros((3, self.numParticles))
            for p in range(self.numParticles):
                positions[:,p] = self.particles[p][:,int(self.timestep)]
        else:
            positions = np.zeros((self.observation_size, self.numParticles))
            for p in range(self.numParticles):
                positions[:,p] = self._observe(self.particles[p][:,int(self.timestep)])
        return positions
        
    def observeTrueState(self):
        if len(self.observations) > 0:
            if self.observations[-1][1] == self.timestep:
                return self._observe(self.observations[-1][0])
        
        obs = np.random.multivariate_normal(self.particles[self.obs_index][:,int(self.timestep)], np.eye(3)*self.observation_variance)
        self.observations.append((obs,self.timestep))
        return self._observe(obs)
    
    
    def _lorenz63_(self, x0):
        x = self.sigma*(x0[1] - x0[0])
        y = x0[0]*(self.rho - x0[2]) - x0[1]
        z = x0[0]*x0[1] - self.beta*x0[2]
        return np.array([x, y, z])
    
    def step(self, t):
        
        k_end = min(self.timestep + t, self.max_timesteps-1)
        for p in self.particles:
            for k in range(int(self.timestep), int(k_end)):
                model_error = np.random.multivariate_normal(np.zeros(3), np.eye(3)*self.model_error)
                
                # Forward Euler:
                #p[:, k+1] = p[:,k] + self.dt*self._lorenz63_(p[:,k]) + model_error
                
                # RK4:
                k1 = self._lorenz63_(p[:,k])
                k2 = self._lorenz63_(p[:,k] + k1*self.dt/2.0)
                k3 = self._lorenz63_(p[:,k] + k2*self.dt/2.0)
                k4 = self._lorenz63_(p[:,k] + k3*self.dt)
                p[:,k+1] = p[:,k] + self.dt*(k1 + 2*k2 + 2*k3 + k4)/6.0
                
        self.timestep = k_end
        self.t = k_end*self.dt
                
    def getDistances(self, obs=None):
        
        pos = self.observeParticles()
        obs = self.observeTrueState()
        dist = np.zeros(self.numParticles)
        for i in range(self.numParticles):
            dist[i] = np.sqrt(np.linalg.norm(pos[:,i]-obs))
            #dist[i] = np.sqrt((pos[:,i]-obs[:])**2 + (pos[1,i]-obs[1])**2 + (pos[2,i]-obs[2])**2  )
        return dist
    
       
    def resample(self, newSampleIndices, reinitialization_variable_notused):
        oldPositions = self.observeParticles(all=True).copy()

        # Make sure to make a clean copy of first resampled particle, and add a disturbance of the next ones.
        resampledOnce = np.full(self.getNumParticles(), False, dtype=bool)
        var = np.eye(3)*self.init_error
        for i in range(self.numParticles):
            index = newSampleIndices[i]
            if resampledOnce[index]:
                self.particles[i][:,self.timestep] = np.random.multivariate_normal(oldPositions[:, index], var)
            else:
                self.particles[i][:,self.timestep] = oldPositions[:, index]
                resampledOnce[index] = True

        
        
        
    def getEnsembleMean(self):
        mean = np.zeros_like(self.particles[0])
        for j in range(self.numParticles):
            mean += self.particles[j]
        mean = mean/self.numParticles
        return mean
    
    def _plot_index(self, i, mean):
        particle_number = 0
        
        t = range(0, self.max_timesteps)
        for p in self.particles:
            if particle_number != self.obs_index:
                plt.plot(t, p[i,:], color="0.8" )
            else:
                plt.plot(t,p[i,:], 'g')
            particle_number += 1
        plt.plot(t, mean[i, :], 'r' )
        if self.obs_vars[i] == '1':
            for o in range(len(self.observations)):
                plt.plot(ensemble.observations[o][1], ensemble.observations[o][0][i], 'bo')
    
    def plotDistanceInfo(self, title=None, obs=None):
        mean = self.getEnsembleMean()
        fig = plt.figure(figsize=(12,8))
        plt.subplot(3,1,1)    
        self._plot_index(0, mean)
        plt.subplot(3,1,2)    
        self._plot_index(1, mean)
        plt.subplot(3,1,3)    
        self._plot_index(2, mean)
        
            
    def enforceBoundaryConditions(self):
        self.sim.drifters.enforceBoundaryConditions()
    
    ### Duplication of code
    def getDomainSizeX(self):
        pass
    def getDomainSizeY(self):
        pass
    def getObservationVariance(self):
        return self.observation_variance
    def getNumParticles(self):
        return self.numParticles
    def getObsMatrix(self):
        return self.obs_matrix
    


In [ ]:
def runAndResample(ensemble, timesteps):
    ensemble.step(timesteps)
    notUsed = 1000
    dautils.stochasticUniversalSampling(ensemble, notUsed)
    print ensemble.observeTrueState()
    


T = 1000
numParticles = 200
obs_vars = '110'
ensemble = Lorenz63Ensemble(cl_ctx, numParticles, T=T, obs_vars=obs_vars)
                            #obs_matrix_diag=obsmat)
                            #model_error=0, init_error=0.1)
                            #observation_variance=5)

obs_frequency = 25
for i in range(int(T/obs_frequency)-5):
    runAndResample(ensemble, obs_frequency)
ensemble.step(1000)
ensemble.plotDistanceInfo()
print ensemble.getObsMatrix()

In [ ]:
o = ensemble.observeTrueState()
print o
H = ensemble.getObsMatrix()
print H
print np.dot(H,o)

In [ ]:
full_o = np.array([1,2,3])
obs_vars = '111'
obs_matrix_array = []
if obs_vars[0]=='1':
    obs_matrix_array.append([1, 0, 0])
if obs_vars[1]=='1':
    obs_matrix_array.append([0,1,0])
if obs_vars[2]=='1':
    obs_matrix_array.append([0,0,1])
H =  np.matrix(obs_matrix_array)
print np.dot(H, full_o)
print str.count(obs_vars, '1')

In [ ]:
a = np.array([1,1,1,1,1,1])
b = np.array([10,10,10,10,0,0])
np.sqrt(np.dot(a, b))
np.linalg.norm(a), np.sqrt(6)